In [ ]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, OWL, XSD
import csv

# Define namespaces
EX = Namespace("http://example.org/owlshelves#")

# Create an RDF graph
g = Graph()

# Bind the namespaces
g.bind("ex", EX)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)


g.add((EX.Book, RDF.type, OWL.Class))
g.add((EX.User, RDF.type, OWL.Class))
g.add((EX.Rating, RDF.type, OWL.Class))


g.add((EX.ratingUser, RDF.type, OWL.ObjectProperty))
g.add((EX.ratingUser, RDFS.domain, EX.Rating))
g.add((EX.ratingUser, RDFS.range, EX.User))

g.add((EX.ratedBook, RDF.type, OWL.ObjectProperty))
g.add((EX.ratedBook, RDFS.domain, EX.Rating))
g.add((EX.ratedBook, RDFS.range, EX.Book))

g.add((EX.ratingscore, RDF.type, OWL.DatatypeProperty))
g.add((EX.ratingscore, RDFS.domain, EX.Rating))
g.add((EX.ratingscore, RDFS.range, XSD.integer))


g.add((EX.hasUserID, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasUserID, RDFS.domain, EX.User))
g.add((EX.hasUserID, RDFS.range, XSD.string))

g.add((EX.hasAge, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasAge, RDFS.domain, EX.User))
g.add((EX.hasAge, RDFS.range, XSD.float))

g.add((EX.hasAgeGroup, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasAgeGroup, RDFS.domain, EX.User))
g.add((EX.hasAgeGroup, RDFS.range, XSD.string))

g.add((EX.hasCountry, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasCountry, RDFS.domain, EX.User))
g.add((EX.hasCountry, RDFS.range, XSD.string))

g.add((EX.hasISBN, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasISBN, RDFS.domain, EX.Book))
g.add((EX.hasISBN, RDFS.range, XSD.string))

g.add((EX.hasTitle, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasTitle, RDFS.domain, EX.Book))
g.add((EX.hasTitle, RDFS.range, XSD.string))

g.add((EX.hasAuthor, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasAuthor, RDFS.domain, EX.Book))
g.add((EX.hasAuthor, RDFS.range, XSD.string))

g.add((EX.hasPublisher, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasPublisher, RDFS.domain, EX.Book))
g.add((EX.hasPublisher, RDFS.range, XSD.string))

g.add((EX.hasGenre, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasGenre, RDFS.domain, EX.Book))
g.add((EX.hasGenre, RDFS.range, XSD.string))

g.add((EX.hasYearOfPublication, RDF.type, OWL.DatatypeProperty))
g.add((EX.hasYearOfPublication, RDFS.domain, EX.Book))
g.add((EX.hasYearOfPublication, RDFS.range, XSD.float))

# Function to add book data to the graph
def add_book_to_graph(row):
    book_uri = EX[row["ISBN"]]
    g.add((book_uri, RDF.type, EX.Book))
    g.add((book_uri, EX.hasISBN, Literal(row["ISBN"], datatype=XSD.string)))
    g.add((book_uri, EX.hasTitle, Literal(row["Book_Title"], datatype=XSD.string)))
    g.add((book_uri, EX.hasAuthor, Literal(row["Book_Author"], datatype=XSD.string)))
    g.add((book_uri, EX.hasYearOfPublication, Literal(row["Year_Of_Publication"], datatype=XSD.float)))
    g.add((book_uri, EX.hasPublisher, Literal(row["Publisher"], datatype=XSD.string)))

# Function to add user data to the graph
def add_user_to_graph(row):
    user_uri = EX[row["User-ID"]]
    g.add((user_uri, RDF.type, EX.User))
    g.add((user_uri, EX.hasUserID, Literal(row["User-ID"], datatype=XSD.string)))
    g.add((user_uri, EX.hasAge, Literal(float(row["Age"]), datatype=XSD.float)))
    g.add((user_uri, EX.hasAgeGroup, Literal(row["Age_group"], datatype=XSD.string)))
    g.add((user_uri, EX.hasCountry, Literal(row["Country"], datatype=XSD.string)))

# Function to add rating data to the graph
def add_rating_to_graph(row):
    rating_uri = EX[f"rating_{row['User-ID']}_{row['ISBN']}"]
    g.add((rating_uri, RDF.type, EX.Rating))
    g.add((rating_uri, EX.ratingUser, EX[row["User-ID"]]))
    g.add((rating_uri, EX.ratedBook, EX[row["ISBN"]]))
    g.add((rating_uri, EX.ratingscore, Literal(row["Book-Rating"], datatype=XSD.integer)))

# Read and process books.csv
with open('books.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        add_book_to_graph(row)

# Read and process users.csv
with open('users.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        add_user_to_graph(row)

# Read and process ratings.csv
with open('ratings.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        add_rating_to_graph(row)

# Serialize the graph to Turtle format and save to a file
g.serialize(destination='owlshelves.ttl', format='turtle')

print("RDF data has been written to output.ttl")